In [ ]:
import subprocess
import matplotlib.pyplot as plt
import numpy as np
from numpy import pi as π
import tqdm
import pygmsh
import firedrake
from firedrake import (
    inner,
    grad,
    dx,
    ds,
    derivative,
    NonlinearVariationalProblem,
    NonlinearVariationalSolver,
)
import irksome
from icepack2 import model, solvers
from icepack2.constants import (
    glen_flow_law as n, gravity as g, ice_density as ρ_I, water_density as ρ_W
)

In [ ]:
with firedrake.CheckpointFile("gibbous.h5", "r") as chk:
    mesh = chk.load_mesh()
    h = chk.load_function(mesh, "thickness")
    u = chk.load_function(mesh, "velocity")
    M = chk.load_function(mesh, "stress")

Q = h.function_space()
V = u.function_space()
Σ = M.function_space()

Z = V * Σ
z = firedrake.Function(Z)
z.sub(0).assign(u)
z.sub(1).assign(M);

In [ ]:
R = 200e3

inlet_angles = π * np.array([-3 / 4, -1 / 2, -1 / 3, -1 / 6])
inlet_widths = π * np.array([1 / 8, 1 / 12, 1 / 24, 1 / 12])

x = firedrake.SpatialCoordinate(mesh)

u_in = 300
h_in = 350
hb = 100
dh, du = 400, 250

hs, us = [], []
for θ, ϕ in zip(inlet_angles, inlet_widths):
    x0 = R * firedrake.as_vector((np.cos(θ), np.sin(θ)))
    v = -firedrake.as_vector((np.cos(θ), np.sin(θ)))
    L = inner(x - x0, v)
    W = x - x0 - L * v
    Rn = 2 * ϕ / π * R
    q = firedrake.max_value(1 - (W / Rn) ** 2, 0)
    hs.append(hb + q * ((h_in - hb) - dh * L / R))
    us.append(firedrake.exp(-4 * (W / R) ** 2) * (u_in + du * L / R) * v)

h_expr = firedrake.Constant(hb)
for h_ in hs:
    h_expr = firedrake.max_value(h_, h_expr)

u_expr = sum(us)

h0 = firedrake.Function(Q).interpolate(h_expr)
u0 = firedrake.Function(V).interpolate(u_expr)

In [ ]:
ε_c = firedrake.Constant(0.01)
τ_c = firedrake.Constant(0.1)

u, M = firedrake.split(z)
fields = {
    "velocity": u,
    "membrane_stress": M,
    "thickness": h,
}

fns = [model.viscous_power, model.ice_shelf_momentum_balance]

rheology = {
    "flow_law_exponent": n,
    "flow_law_coefficient": ε_c / τ_c ** n,
}

L = sum(fn(**fields, **rheology) for fn in fns)
F = derivative(L, z)

In [ ]:
h_min = firedrake.Constant(0.1)
rfields = {
    "velocity": u,
    "membrane_stress": M,
    "thickness": firedrake.max_value(h_min, h),
}

L_r = sum(fn(**rfields, **rheology) for fn in fns)
F_r = derivative(L_r, z)
H_r = derivative(F_r, z)

In [ ]:
prognostic_problem = model.mass_balance(
    thickness=h,
    velocity=u,
    accumulation=firedrake.Constant(0.0),
    thickness_inflow=h0,
    test_function=firedrake.TestFunction(Q),
)

final_time = 400.0
num_steps = 400

dt = firedrake.Constant(final_time / num_steps)
t = firedrake.Constant(0.0)
method = irksome.BackwardEuler()
prognostic_params = {
    "solver_parameters": {
        "snes_type": "ksponly",
        "ksp_type": "gmres",
        "pc_type": "bjacobi",
    },
}
prognostic_solver = irksome.TimeStepper(
    prognostic_problem, method, t, dt, h, **prognostic_params
)

In [ ]:
bc = firedrake.DirichletBC(Z.sub(0), firedrake.Constant((0, 0)), (1,))
problem = solvers.ConstrainedOptimizationProblem(L, z, H=H_r, bcs=bc)
diagnostic_solver = solvers.NewtonSolver(problem, tolerance=1e-4)

In [ ]:
residuals = [diagnostic_solver.solve()]

In [ ]:
radius = firedrake.Constant(60e3)
y = firedrake.Constant((0.0, radius))
mask = firedrake.conditional(inner(x - y, x - y) < radius**2, 0.0, 1.0)

In [ ]:
calving_frequency = 24.0
time_since_calving = 0.0

for step in tqdm.trange(num_steps):
    prognostic_solver.advance()

    if time_since_calving > calving_frequency:
        h.interpolate(mask * h)
        time_since_calving = 0.0
    time_since_calving += float(dt)
    h.interpolate(firedrake.max_value(0, h))

    residuals.append(diagnostic_solver.solve())

In [ ]:
ts = np.linspace(0, final_time, num_steps + 1)
fig, ax = plt.subplots()
ax.bar(ts, [len(r) for r in residuals]);